# Python Water Quality Analysis: Elemental Concentration in the Friant-Kern Canal


Hello! My name is Cayleigh, and I am a chemical engineering student at Clovis Community College. This notebook is an independent data analysis project from what I've been working on with a research group on water quality in the Friant-Kern Canal. We're looking to investigate metal concentrations, with special regard for environmental hazard and agricultural runoff. I'm analyzing my research group's data collected from Octoberthrough May(minus Nov and Dec).

This is a segment of my effort to gain experience in environmental chemistry, data science, and community-impact research. I am also studying Python and MATLAB independently to further understand scientific data and advance my competence in experimental analysis. 

Below shows parts of my process—cleaning, visualizing, and examines seasonal trends, visualize concentration distributions, and assess potential health risks, with a focus on Thallium (Tl).

In [ ]:
# 1. Imports and Setup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')# I chose 'whitegrid' for better visibility of the data points(stat plot); 
#I want to focus on distribution and relationships in the data not really the aesthetics. 

## 2. Load and Label Monthly Data

In [ ]:
# Load monthly data (update paths as needed) Skipped December and November due to insufficent data. 
# May dataset was my first dataset I plotted, that is why it's xlsx is Kern_Data
# Dictionary that maps month names to filenames
months = {
    'October': 'Oct2024_Data.xlsx',
    'January': 'Jan2025_Data.xlsx',
    'February': 'Feb2025_Data.xlsx',
    'March': 'March2025_Data.xlsx',
    'April': 'April2025_Data.xlsx',
    'May': 'Kern_Data.xlsx'
}

# Loop through each month to clean the data from uneccessary columns and rows, and adds a Month column to each dataframe
dataframes = [] #[] collects clean data from each month
for month, filename in months.items():
    df = pd.read_excel(f'../data/{filename}')  # Adjust path if needed
    df.columns = df.columns.str.strip()
    df['Month'] = month
    dataframes.append(df) 
    #gave me a hard time understanding I needed to append the dataframes to a list, 
    #so I can combine them later

# COmbines all data into a single dataframe. def:concat()= cancatenate/combine multiple dataframes into one 
all_data = pd.concat(dataframes, ignore_index=True)

## 3. Clean Data

In [ ]:
# Had to convert the concentration(ppm) to all float/integer values and also remove rows with NaN/No values in the conc column
all_data['Concentration'] = pd.to_numeric(all_data['Concentration'], errors='coerce')
all_data = all_data.dropna(subset=['Concentration'])#Drops anything that isnt a number

# Remove diluted and non-field samples
# #Lims ID column is the type of sample. My research group labeled samples based on what road we collected it from
#or if it is a standard sample(DI or NO3.) 
all_data['Lims ID'] = all_data['Lims ID'].astype(str)  
all_data = all_data[~all_data['Lims ID'].str.contains(r'\d+\.\d+', regex=True)]
#Decided to remove any Lims ID with a decimal(.) in it to take out the diluted 
# samples(1.5, 2.5, 3.5) which my research group tested for precision.
all_data = all_data[~all_data['Lims ID'].str.contains('DI|HNO', case=False, na=False)]
all_data['Location'] = all_data['Lims ID'].str.extract(r'([A-Za-z\s]+)', expand=False).str.strip() # Extract clean location name

## 4. Heatmap of Top Elements by Location

In [ ]:
# Here, I sort the water quality data by element and location and calculate
# the average concentration of each metal at each sampling site. Then I focus on
# the top 10 elements with the largest overall average concentrations to limit the significant metals
# of interest.

element_avg = all_data.groupby(['Element Label', 'Location'])['Concentration'].mean().reset_index()
top_elements = (all_data.groupby('Element Label')['Concentration']
                .mean().sort_values(ascending=False).head(10).index)
element_avg = element_avg[element_avg['Element Label'].isin(top_elements)]


# To visualize this data, I create a heatmap showing how these top elements vary by location.
# This enables me to easily spot patterns or hotspots of metal concentration along the canal, which
# may signify potential sources of contamination or areas needing additional monitoring. I havent
# fully analyzed the data or effects of  high conc. metals on crop growth which I'd like to explore in the future. 
heatmap_data = element_avg.pivot(index='Element Label', columns='Location', values='Concentration')

plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, cmap="viridis", annot=True, fmt=".3f")
plt.title('Average Concentration of Top Elements by Location')
plt.ylabel('Element')
plt.xlabel('Location')
plt.tight_layout()
plt.show() #I want to learn how to create a dropdown menu to select the month and plot of the data for that month. 

## 5. Boxplot of Element Concentrations (Log Scale)

In [ ]:
# Here, I learned how to use a boxplot to examine the distribution of concentrations for the major elements
# for all samples. This plot indicates to me the spread and variability
# count of each metal's concentration, including outliers.

plt.figure(figsize=(14, 6))
sns.boxplot(data=all_data[all_data['Element Label'].isin(top_elements)],
            x='Element Label', y='Concentration')
plt.yscale('log')
plt.title('Distribution of Element Concentrations')
plt.xlabel('Element')
plt.ylabel('Concentration (ppm)')
plt.tight_layout()
plt.show()
# As metal concentrations may vary widely, I put a log scale on the y-axis
# To better capture and contrast values that differ by orders of magnitude.

## 6. Thallium (Tl) Concentration Focus

In [ ]:
# I'm aiming at Thallium (Tl) as my mentor found this concentration most alarming/suprising in Kern canal
# it's a poisonous metal of interest in water quality.
# I first exclude the data to keep only those samples that are labeled as 'Tl'.
# I then exclude any laboratory control/standard or reagent blank (samples with 'DI' or 'HNO') in order to keep
# the analysis strictly to actual field samples.

tl_data = all_data[all_data['Element Label'] == 'Tl'].copy()
tl_data = tl_data[~tl_data['Lims ID'].str.contains('DI|HNO', case=False, na=False)]
tl_data['Location'] = tl_data['Lims ID'].str.extract(r'([A-Za-z\s]+)', expand=False).str.strip()
# I next extract the common location name from the sample ID in order to group data appropriately.
# Average Tl by location and month
 #I calculate the average concentration of Thallium for each location and month,
# and sort the results to find out where the locations with highest concentrations are.
tl_avg = tl_data.groupby(['Location', 'Month'])['Concentration'].mean().reset_index()
tl_avg_sorted = tl_avg.sort_values('Concentration', ascending=False)

In [ ]:
# Now I wanted to plot the comparison of Thallium levels across all the sites to possibly discover 
# if there is a location source where Tl is more concentrated
# I use a bar plot to show the **mean Thallium concentration** for each site,
# in the order from highest to lowest. This helps me to identify the areas
# where Thallium pollution might be an issue.

plt.figure(figsize=(12, 6))
sns.barplot(data=tl_avg_sorted, x='Location', y='Concentration')
plt.title('Average Thallium Concentration by Location (Highest to Lowest)')
plt.xlabel('Location')
plt.ylabel('Average Concentration (ppm)')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
# Bar plot with EPA limit
plt.figure(figsize=(12, 6))
sns.barplot(data=tl_avg, x='Location', y='Concentration', hue='Month')
plt.axhline(y=0.002, color='red', linestyle='--', linewidth=1, label='EPA Limit (0.002 ppm)')
plt.title('Thallium Concentration by Location and Month')
plt.xlabel('Location')
plt.ylabel('Concentration (ppm)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()